In [1]:
import pandas as pd

In [2]:
myfile="data/pop_and_death.txt"
df = pd.read_csv(myfile,sep='\t',header=(0), skipfooter=56)
df.head(5)

C:\Users\ama29\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


,Notes,State,State Code,Five-Year Age Groups,Five-Year Age Groups Code,Year,Year Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1,< 1 year,1,1999,1999,606,58787,1030.8
1,NaN,Alabama,1,< 1 year,1,2000,2000,596,59101,1008.4
2,NaN,Alabama,1,< 1 year,1,2001,2001,567,61537,921.4
3,NaN,Alabama,1,< 1 year,1,2002,2002,539,58963,914.1
4,NaN,Alabama,1,< 1 year,1,2003,2003,519,58021,894.5


In [3]:
#get values of Five Year Age Groups for recoding
df['Five-Year Age Groups'].value_counts()

60-64 years     1020
35-39 years     1020
20-24 years     1020
15-19 years     1020
50-54 years     1020
45-49 years     1020
< 1 year        1020
75-79 years     1020
25-29 years     1020
85-89 years     1020
90-94 years     1020
80-84 years     1020
55-59 years     1020
65-69 years     1020
70-74 years     1020
40-44 years     1020
95-99 years     1020
30-34 years     1020
100+ years      1016
1-4 years        932
10-14 years      901
5-9 years        850
Not Stated       111
Name: Five-Year Age Groups, dtype: int64

In [4]:
#recode age variables: 0-24, 25-34, 35-44, 45-54, 55 plus
def rename_age(ag):
    if(ag=="< 1 year")|(ag=="1-4 years")|(ag=="5-9 years")|(ag=="10-14 years")|(ag=="15-19 years")|(ag=="20-24 years"):
        return "0-24"
    elif (ag=="25-29 years")|(ag=="30-34 years"):
        return "25-34"
    elif (ag=="35-39 years")|(ag=="40-44 years"):
        return "35-44"
    elif (ag=="45-49 years")|(ag=="50-54 years"):
        return "45-54"
    elif (ag=="Not Stated"):
        return "NS"
    else:
        return "55+"

#run function to recode age groups
df['Age_Group'] = df['Five-Year Age Groups'].apply(rename_age)

#repressed cells due to low sample sizes are coded as NA or Unavilalbe. Recode these values to 0 so they are not included in our population
def recode_missing(n):
    if(n=="Unavailable")|(n=="Not Applicable"):
        return("0")
    else:
        return(n)

#run recode fuction and convert to int
df['death_num']=df['Deaths'].apply(recode_missing).astype('int') 
df['pop_num']=df['Population'].apply(recode_missing).astype('int')

#check conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Notes                      0 non-null      float64
 1   State                      22170 non-null  object 
 2   State Code                 22170 non-null  int64  
 3   Five-Year Age Groups       22170 non-null  object 
 4   Five-Year Age Groups Code  22170 non-null  object 
 5   Year                       22170 non-null  int64  
 6   Year Code                  22170 non-null  int64  
 7   Deaths                     22170 non-null  int64  
 8   Population                 22170 non-null  object 
 9   Crude Rate                 22170 non-null  object 
 10  Age_Group                  22170 non-null  object 
 11  death_num                  22170 non-null  int32  
 12  pop_num                    22170 non-null  int32  
dtypes: float64(1), int32(2), int64(4), object(6)
m

In [5]:
#check recoded age variable
df['Age_Group'].value_counts()

55+      10196
0-24      5743
35-44     2040
25-34     2040
45-54     2040
NS         111
Name: Age_Group, dtype: int64

In [6]:
#drop unnecssary columns
clean_df=(df.drop(columns=['Notes','State Code','Five-Year Age Groups','Deaths','Population','Five-Year Age Groups Code','Year Code','Crude Rate']).dropna(how='any'))


In [7]:
#first sort then group the cleaned data by Age Group, State, and Year.
group_vars=['Age_Group','State','Year']
sort_df=clean_df.sort_values(group_vars)

group_df=sort_df.groupby(group_vars)

In [8]:
#Sum deaths and pouplation by age group and put them into a new data frame. This dataframe becomes our data for analysis. 
deaths=group_df['death_num'].sum()
pop=group_df['pop_num'].sum()

summ_df=pd.DataFrame({'deaths':deaths,
         'population':pop})
print(summ_df)

                        deaths  population
Age_Group State   Year                    
0-24      Alabama 1999    1663     1562039
                  2000    1539     1563034
                  2001    1532     1568372
                  2002    1513     1567247
                  2003    1446     1569632
...                        ...         ...
NS        Texas   2003      40           0
                  2004      27           0
                  2005      35           0
                  2007      14           0
                  2014      11           0

[5211 rows x 2 columns]


In [9]:
#Export the clean data to CSV. This is the file to use for analys.
#Note: exporting to CSV and then importing removes any issues caused by grouping the data. I added this step 
#as I was working on pivoting the file, I was haivng issues trying to pivot a grouped data file.  Exporitng to CSV 
#and bringing it backed removed the group settings
summ_df.to_csv('data/clean_pop.csv')

In [10]:
#read the clean data file
clean_df=pd.read_csv('data/clean_pop.csv')
clean_df.head()

,Age_Group,State,Year,deaths,population
0,0-24,Alabama,1999,1663,1562039
1,0-24,Alabama,2000,1539,1563034
2,0-24,Alabama,2001,1532,1568372
3,0-24,Alabama,2002,1513,1567247
4,0-24,Alabama,2003,1446,1569632
